In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from StatusMsg import StatusMsg
import json
import os
import io
import datetime
import timedelta

In [2]:
def india(state,date):
    # URL = "https://www.mygov.in/corona-data/covid19-statewise-status/"
    # file_name, headers = urllib.request.urlretrieve(URL)
    soup = BeautifulSoup(open("../INPUT/"+date+"/TT_State.html", encoding="utf8"), "html.parser")

    # sum_URL = "https://www.mygov.in/covid-19"
    # sum_file_name, sum_headers = urllib.request.urlretrieve(sum_URL)
    sum_soup = BeautifulSoup(open("../INPUT/"+date+"/TT.html", encoding="utf8"), "html.parser")

    STATES = soup.find_all("div", {"class": "field field-name-field-select-state field-type-list-text field-label-above"})
    CONFIRMED = soup.find_all("div", {"class": "field field-name-field-total-confirmed-indians field-type-number-integer field-label-above"})
    CURED_DISCHARGED = soup.find_all("div", {"class": "field field-name-field-cured field-type-number-integer field-label-above"})
    DEATH = soup.find_all("div", {"class": "field field-name-field-deaths field-type-number-integer field-label-above"})

    states = []
    for val in STATES:
        states.append(str(val.getText()).split(":")[1].lstrip().replace("Telengana", "Telangana").replace('Andaman And Nicobar', 'Andaman and Nicobar Islands').replace('Andaman and Nicobar','Andaman and Nicobar Islands'))

    confirmed = []
    for val in CONFIRMED:
        confirmed.append(str(val.getText()).split(":")[1].lstrip())

    cured = []
    for val in CURED_DISCHARGED:
        cured.append(str(val.getText()).split(":")[1].lstrip())

    death = []
    for val in DEATH:
        death.append(str(val.getText()).split(":")[1].lstrip())

    states_data = pd.DataFrame(list(zip(states, confirmed, cured, death)))

    final_df_col = [
        'Date', 'State/UTCode', 'District', 'tested_last_updated_district', 'tested_source_district',
        'notesForDistrict', 'cumulativeConfirmedNumberForDistrict', 'cumulativeDeceasedNumberForDistrict',
        'cumulativeRecoveredNumberForDistrict', 'cumulativeTestedNumberForDistrict',
        'cumulativeVaccinatedNumberForDistrict', 'last_updated', 'tested_last_updated_state', 'tested_source_state',
        'notesForState', 'cumulativeConfirmedNumberForState', 'cumulativeDeceasedNumberForState',
        'cumulativeRecoveredNumberForState', 'cumulativeTestedNumberForState', 'cumulativeVaccinatedNumberForState'
    ]

    states_data = states_data.rename(
        columns={0: "District", 1: "cumulativeConfirmedNumberForDistrict", 2: "cumulativeRecoveredNumberForDistrict",
                 3: "cumulativeDeceasedNumberForDistrict"})

    states_data["Date"] = date#str(datetime.datetime.now().date())

    states_data["State/UTCode"] = state

    states_data['cumulativeConfirmedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "t_case"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))

    states_data['cumulativeDeceasedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "death_case"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))

    states_data['cumulativeRecoveredNumberForState'] = int(
        sum_soup.findAll("div", {"class": "discharge"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))
    
    
    states_data["last_updated"] = str(datetime.datetime.now())

    states_data['cumulativeTestedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "testing_result"})[0].findAll("strong")[0].getText().replace(",", ""))
    
    
    states_data['tested_last_updated_state'] = sum_soup.findAll(
        "div", {"class": "test_title"})[0].getText().strip().split("up to ")[-1]
    
    states_data['deltaTestedForState'] = int(
        sum_soup.findAll("div", {"class": "testing_sample"})[0].findAll("strong")[0].getText().replace(",", ""))
    
    for vcount in sum_soup.findAll("div", {"class": "total-vcount"}):
        if 'yday' not in vcount.attrs['class']:
            states_data['cumulativeVaccinatedNumberForState'] = int(vcount.findAll("strong")[0].getText().replace(",", ""))

    # states_data = states_data.reindex(columns=final_df_col)
    return states_data

In [3]:
today = (datetime.datetime.now() - timedelta.Timedelta(days=0)).date()
pDate = (datetime.datetime.now() - timedelta.Timedelta(days=1)).date()
cowinDate = (datetime.datetime.now() - timedelta.Timedelta(days=2)).date()
print(str(today))
print(str(pDate))

TT_df = india("TT",str(today))

# ASHA .. For Running previous dates
# date_time_str = '2021/11/06'
# date_time_obj = datetime.datetime.strptime(date_time_str, '%Y/%m/%d')
# print(str(date_time_obj.date()))
# today = (date_time_obj - timedelta.Timedelta(days=0)).date()
# pDate = (date_time_obj - timedelta.Timedelta(days=1)).date()
# print(str(today))
# print(str(pDate))
# TT_df = india("TT",str(today))
# ASHA .. For Running previous dates



2022-02-05
2022-02-04


In [4]:
if not os.path.isdir(os.path.join("..","RAWCSV",str(pDate),"myGov")):
    os.mkdir(os.path.join("..","RAWCSV",str(pDate),"myGov"))
    print("Created")

Created


In [5]:
#mygov of prevDate
from ExtractStateMyGov import ExtractStateMyGov
source = pd.read_csv(r"../sources.csv")
for idx in source.index:
    if source["StateCode"][idx] != "TT":
        print(source["StateCode"][idx])
        if source["myGov"][idx] == "yes":
            if source["StateCode"][idx] == "DL":
                temp_df = ExtractStateMyGov(source["StateCode"][idx],str(today))
                df_addTest = pd.read_csv("../INPUT/DL_Tested.csv")
                temp_df['Tested'] = df_addTest[df_addTest["Date"] == str(pDate)]["Cumulative_Tested"].item()
            else:
                temp_df = ExtractStateMyGov(source["StateCode"][idx],str(today))
        else:
            temp_df = ExtractStateMyGov(source["StateCode"][idx],str(today), no_source = True)
        temp_df["Date"] = pDate
        temp_df.to_csv(os.path.join("..","RAWCSV",str(pDate),"myGov",source["StateCode"][idx]+"_raw.csv"))

AP
AR
AS


/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df["cumulativeConfirmedNumberForState"] = state_df['cumulativeConfirmedNumberForDistrict'][state_df['District'] == 'Unknown'] = int(ind["cumulativeConfirmedNumberForDistrict"][ind["District"].str.contains(state_name)].values[0])
/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df["cumulativeDeceasedNumberForState"] = state_df['cumulativeDeceasedNumberForDistrict'][state_df['District'] == 'Unknown'] = int(ind["cumulativeDeceasedNumberForDis

BR
CT
GA
GJ
HR
HP
JH
KA
KL
MP
MH
MN
ML
MZ
NL


/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][df['District'] == district] = None


OR
PB
RJ
SK
TN
TG
TR
UP
UT
WB
AN
CH
DN
DL
LA
LD
JK
PY


In [6]:
# TT_df = pd.read_csv("../RAWCSV/2021-10-31/TT_raw.csv")

In [7]:
TT_df = TT_df.dropna(1)

In [8]:
import urllib.request, json 
with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(pDate)) as url:
    data = json.loads(url.read().decode())

In [9]:
#pDate
#tot_pd
#precaution_dose
# data

In [10]:
TT_df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
TT_df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
TT_df["cumulativeVaccinated3NumberForState"] = data["topBlock"]["vaccination"]["tot_pd"]
TT_df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]

In [11]:
print(TT_df["cumulativeVaccinated1NumberForState"][0])
print(TT_df["cumulativeVaccinated2NumberForState"][0])
print(TT_df["cumulativeVaccinated3NumberForState"][0])
print(TT_df["cumulativeVaccinatedNumberForState"][0])

949357529
726246043
13701474
1689305046


In [12]:
states = []
vaccinated = []
vaccinated1 = []
vaccinated2 = []
vaccinated3 = []
for sta in data['getBeneficiariesGroupBy']:
    if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
        states.append('Dadra and Nagar Haveli and Daman and Diu')
    else:
        states.append(sta['title'])
    vaccinated.append(sta['total'])
    vaccinated1.append(sta['partial_vaccinated'])
    vaccinated2.append(sta['totally_vaccinated'])
    vaccinated3.append(sta['precaution_dose'])

In [13]:
VAC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2,vaccinated3))).groupby(0, as_index=False).sum()

In [14]:
VAC_df = VAC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",4:"cumulativeVaccinated3NumberForDistrict"})

In [15]:
TT_df = TT_df.merge(VAC_df,how="left",on="District")

In [16]:
TT_df["tested_source_state"] = "https://www.icmr.gov.in/"

In [17]:
# TT_df.to_csv("TT_test.csv")

In [18]:
TT_df["tested_last_updated_state"] = str(datetime.datetime.strptime(TT_df["tested_last_updated_state"][0] , '%b %d, %Y'))

# Generate TT_final

In [19]:
# cs = pd.read_csv("case_time_series.csv")

In [20]:
# cs.columns

In [21]:
# cs.tail()

In [22]:
# import timedelta
# delta_date = str((datetime.datetime.now() - timedelta.Timedelta(days=1)).date())

In [23]:
delta_date = str(pDate)

In [24]:
delta_date

'2022-02-04'

In [25]:
# ind = cs[cs['Date_YMD']==delta_date].index[0]

# Integrate is CS only

In [26]:
# states_df = pd.read_csv("states.csv")

In [27]:
# cs = states_df

In [28]:
# ind = cs[(cs['Date']==delta_date) & (cs["State"] == "India")].index[0]

In [29]:
# ind

# Reading from Raw

In [30]:
cs = pd.read_csv("../RAWCSV/"+delta_date+"/TT_final.csv")

In [31]:
# cs.columns

In [32]:
# TT_df["deltaConfirmedForState"] = TT_df["cumulativeConfirmedNumberForState"] - cs["Confirmed"][ind]
# TT_df["deltaDeceasedForState"] = TT_df["cumulativeDeceasedNumberForState"] - cs["Deceased"][ind]
# TT_df["deltaRecoveredForState"] = TT_df["cumulativeRecoveredNumberForState"] - cs["Recovered"][ind]
TT_df["deltaConfirmedForState"] = TT_df["cumulativeConfirmedNumberForState"] - cs["cumulativeConfirmedNumberForState"][0]
TT_df["deltaDeceasedForState"] = TT_df["cumulativeDeceasedNumberForState"] - cs["cumulativeDeceasedNumberForState"][0]
TT_df["deltaRecoveredForState"] = TT_df["cumulativeRecoveredNumberForState"] - cs["cumulativeRecoveredNumberForState"][0]
TT_df["deltaTestedForState"] = TT_df["cumulativeTestedNumberForState"] - cs["cumulativeTestedNumberForState"][0]

In [33]:
delta_date

'2022-02-04'

In [34]:
import urllib.request, json 
with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(cowinDate)) as url:
    data = json.loads(url.read().decode())

In [35]:
# TT_df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
# TT_df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
# TT_df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]

In [36]:
print(data["topBlock"]["vaccination"]["tot_pd"])
print(data["topBlock"]["vaccination"]["tot_dose_2"])
print(data["topBlock"]["vaccination"]["tot_dose_1"])
print(data["topBlock"]["vaccination"]["total_doses"])

13297557
722945642
948110180
1684353379


In [37]:
TT_df["deltaVaccinated3ForState"] = TT_df["cumulativeVaccinated3NumberForState"] - data["topBlock"]["vaccination"]["tot_pd"]
TT_df["deltaVaccinated2ForState"] = TT_df["cumulativeVaccinated2NumberForState"] - data["topBlock"]["vaccination"]["tot_dose_2"]
TT_df["deltaVaccinated1ForState"] = TT_df["cumulativeVaccinated1NumberForState"] - data["topBlock"]["vaccination"]["tot_dose_1"]
TT_df["deltaVaccinatedForState"] = TT_df["cumulativeVaccinatedNumberForState"] - data["topBlock"]["vaccination"]["total_doses"]

In [38]:
print(TT_df["deltaVaccinated3ForState"][0])
print(TT_df["deltaVaccinated1ForState"][0])
print(TT_df["deltaVaccinated2ForState"][0])
print(TT_df["deltaVaccinatedForState"][0])

403917
1247349
3300401
4951667


In [39]:
# TT_df["deltaVaccinatedForState"]

In [40]:
states = []
vaccinated = []
vaccinated1 = []
vaccinated2 = []
vaccinated3 = []
for sta in data['getBeneficiariesGroupBy']:
    if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
        states.append('Dadra and Nagar Haveli and Daman and Diu')
    else:
        states.append(sta['title'])
    vaccinated.append(sta['total'])
    vaccinated1.append(sta['partial_vaccinated'])
    vaccinated2.append(sta['totally_vaccinated'])
    vaccinated3.append(sta['precaution_dose'])

In [41]:
prevdayVACC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2,vaccinated3))).groupby(0, as_index=False).sum()

In [42]:
prevdayVACC_df = prevdayVACC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",4:"cumulativeVaccinated3NumberForDistrict"})

In [43]:
lst = ["Vaccinated1","Vaccinated2","Vaccinated3","Vaccinated"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        TT_df["delta{}ForDistrict".format(val)][idx] = TT_df["cumulative{}NumberForDistrict".format(val)][idx] - prevdayVACC_df[prevdayVACC_df["District"] == TT_df["District"][idx]]["cumulative{}NumberForDistrict".format(val)].item()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [44]:
# states_df = pd.read_csv("states.csv")

# No reading states

In [45]:
states_df = cs[["Date","District","cumulativeConfirmedNumberForDistrict","cumulativeRecoveredNumberForDistrict","cumulativeDeceasedNumberForDistrict"]]

In [46]:
# states_df

In [47]:
# deltaConfirmedForDistrict",
#                  "deltaDeceasedForDistrict",
#                  "deltaRecoveredForDistrict",
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        TT_df["delta{}ForDistrict".format(val)][idx] = int(TT_df["cumulative{}NumberForDistrict".format(val)][idx]) - int(states_df[(states_df["District"] == TT_df["District"][idx]) & (states_df["Date"] == delta_date)]["cumulative{}NumberForDistrict".format(val)].item())

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
delta_date

'2022-02-04'

In [49]:
lst = ["Vaccinated1","Vaccinated2","Vaccinated3","Vaccinated"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        TT_df["delta{}ForDistrict".format(val)][idx] = TT_df["cumulative{}NumberForDistrict".format(val)][idx] - prevdayVACC_df[prevdayVACC_df["District"] == TT_df["District"][idx]]["cumulative{}NumberForDistrict".format(val)].item()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [50]:
# TT_df 7dma Cummulative

In [51]:
cs.tail()

,Unnamed: 0,District,cumulativeConfirmedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeDeceasedNumberForDistrict,Date,State/UTCode,cumulativeConfirmedNumberForState,cumulativeDeceasedNumberForState,cumulativeRecoveredNumberForState,last_updated,cumulativeTestedNumberForState,tested_last_updated_state,deltaTestedForState,cumulativeVaccinatedNumberForState,cumulativeVaccinated1NumberForState,cumulativeVaccinated2NumberForState,cumulativeVaccinated3NumberForState,cumulativeVaccinatedNumberForDistrict,cumulativeVaccinated1NumberForDistrict,cumulativeVaccinated2NumberForDistrict,cumulativeVaccinated3NumberForDistrict,tested_source_state,deltaConfirmedForState,deltaDeceasedForState,deltaRecoveredForState,deltaVaccinated3ForState,deltaVaccinated2ForState,deltaVaccinated1ForState,deltaVaccinatedForState,deltaVaccinated1ForDistrict,deltaVaccinated2ForDistrict,deltaVaccinated3ForDistrict,deltaVaccinatedForDistrict,deltaConfirmedForDistrict,deltaRecoveredForDistrict,deltaDeceasedForDistrict,7DmaTestedForState,delta21_14confirmedForState,statePopulation,StateName,districtPopulation
31,31,Telangana,771828,734628,4096,2022-02-04,TT,41952712,500055,40017088,2022-02-04 12:54:33.280659,735804280,2022-02-03 00:00:00,1611666,1684353379,948110180,722945642,13297557,54819731,30381964,24132610,305157,https://www.icmr.gov.in/,149394,1072,246674,430761,4039767,1426319,5896847,38871,151820,6148,196839,2421,3980,2,NaN,NaN,1332900000,Telangana,37220000
32,32,Tripura,100584,97943,915,2022-02-04,TT,41952712,500055,40017088,2022-02-04 12:54:33.280659,735804280,2022-02-03 00:00:00,1611666,1684353379,948110180,722945642,13297557,4993627,2750858,2194247,48522,https://www.icmr.gov.in/,149394,1072,246674,430761,4039767,1426319,5896847,4072,15034,1427,20533,62,703,1,NaN,NaN,1332900000,Tripura,3992000
33,33,Uttar Pradesh,2034456,1969708,23277,2022-02-04,TT,41952712,500055,40017088,2022-02-04 12:54:33.280659,735804280,2022-02-03 00:00:00,1611666,1684353379,948110180,722945642,13297557,266190211,158914606,105677848,1597757,https://www.icmr.gov.in/,149394,1072,246674,430761,4039767,1426319,5896847,584052,765211,59704,1408967,5240,5541,23,NaN,NaN,1332900000,Uttar Pradesh,224979000
34,34,Uttarakhand,426927,394970,7581,2022-02-04,TT,41952712,500055,40017088,2022-02-04 12:54:33.280659,735804280,2022-02-03 00:00:00,1611666,1684353379,948110180,722945642,13297557,16314018,8491167,7541873,280978,https://www.icmr.gov.in/,149394,1072,246674,430761,4039767,1426319,5896847,6528,62082,10105,78715,1618,3306,7,NaN,NaN,1332900000,Uttarakhand,11141000
35,35,West Bengal,2002169,1960300,20723,2022-02-04,TT,41952712,500055,40017088,2022-02-04 12:54:33.280659,735804280,2022-02-03 00:00:00,1611666,1684353379,948110180,722945642,13297557,123314043,69487334,52759707,1067002,https://www.icmr.gov.in/,149394,1072,246674,430761,4039767,1426319,5896847,41724,377404,41251,460379,1916,2614,36,NaN,NaN,1332900000,West Bengal,96906000


In [52]:
# cs = pd.read_csv("case_time_series.csv")

In [53]:
# states_df = pd.read_csv("states.csv")

# Remenber to read CS and states

In [54]:
# from datetime import datetime, timedelta
# lst = ["Confirmed","Recovered","Deceased"]
# for val in lst:
#     value = 0
#     for i in range(7):
#         if i == 0:
#             value = TT_df["cumulative{}NumberForState".format(val)]
#         else:
#             prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
#             ind = cs[cs['Date_YMD']==str(prev_date.date())].index[0]
#             value =+ cs["Total {}".format(val)][ind]
#     TT_df["7Dma{}ForState".format(val)] = value/7

In [55]:
# import urllib.request, json 
# va1 = 0
# va2 = 0
# va = 0
# for i in range(7):
#     prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
#     with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(prev_date.date())) as url:
#         data = json.loads(url.read().decode())
#         va1 += data["topBlock"]["vaccination"]["tot_dose_1"]
#         va2 += data["topBlock"]["vaccination"]["tot_dose_2"]
#         va += data["topBlock"]["vaccination"]["total_doses"]


In [56]:
# TT_df["7DmaVaccinated1ForState"] = va1
# TT_df["7DmaVaccinated2ForState"] = va2
# TT_df["7DmaVaccinatedForState"] = va

In [57]:
# 7dma District

In [58]:
# lst = ["Confirmed","Recovered","Deceased"]
# for val in lst:
#     value = 0
#     for idx in TT_df.index:
#         for i in range(7):
#             if i == 0:
#                 value = int(TT_df["cumulative{}NumberForDistrict".format(val)][0])
#             else:
#                 prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
#                 value += int(states_df[(states_df["State"] == TT_df["District"][idx]) & (states_df["Date"] == str(prev_date.date()))]['{}'.format(val)].item())
#     TT_df["7Dma{}ForDistrict".format(val)] = value/7          

In [59]:
# cowin = pd.read_csv("cowin_vaccine_data_statewise.csv")

In [60]:
# va1 = 0
# va2 = 0
# va = 0
# TT_df["7DmaVaccinatedForDistrict"] = 0
# TT_df["7DmaVaccinated1ForDistrict"] = 0
# TT_df["7DmaVaccinated2ForDistrict"] = 0
# for idx in TT_df.index:
#     for i in range(7):
#         if i == 0:
#             va = int(TT_df["cumulativeVaccinatedNumberForDistrict"][idx])
#             va1 = int(TT_df["cumulativeVaccinated1NumberForDistrict"][idx])
#             va2 = int(TT_df["cumulativeVaccinated2NumberForDistrict"][idx])
#         else:
#             prev_date=datetime.strptime("2021/10/31","%Y/%m/%d")+timedelta(days=(-1*i))
#             va += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["Total Doses Administered"].item())
#             va1 += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["First Dose Administered"].item())
#             va2 += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["Second Dose Administered"].item())
# TT_df["7DmaVaccinatedForDistrict"][idx] = va/7
# TT_df["7DmaVaccinated1ForDistrict"][idx] = va1/7
# TT_df["7DmaVaccinated2ForDistrict"][idx] = va2/7

In [61]:
TT_df["7DmaTestedForState"] = None
TT_df["delta21_14confirmedForState"] = None

In [62]:
population = pd.read_csv("../CSV/StatePopulation.csv")

In [63]:
sources = pd.read_csv("../sources.csv")

In [64]:
population = population.merge(sources,how="left",left_on = "State", right_on = "StateCode")

In [65]:
# population

In [66]:
# population[["StateName","StatePop"]]

In [67]:
TT_df["statePopulation"] = population[population["StateName"] == "India"]["StatePop"].item()

In [68]:
# TT_df

In [69]:
TT_df = TT_df.merge(population[["StateName","StatePop"]],how="left",left_on = "District", right_on = "StateName")

In [70]:
TT_df = TT_df.rename(columns={"StatePop":"districtPopulation"})

In [71]:
today

datetime.date(2022, 2, 5)

In [72]:
TT_df["Date"] = str(today)

In [73]:
TT_df.to_csv('../RAWCSV/'+str(today)+'/TT_final.csv')

In [74]:
TT_df.to_csv('../RAWCSV/'+str(today)+'/TT_final-'+str(today)+'.csv')

In [75]:
TT_df["Date"] = str(pDate)

In [76]:
TT_df.to_csv('../RAWCSV/'+str(pDate)+'/TT_final.csv')

# Stop Here